# Project 1

    We begin by loading the data

In [3]:

%matplotlib inline 
import numpy as np   # generic stuff
import matplotlib.pyplot as plt

#### REMOVE THIS LINE BEFORE SUBMISSION
import pandas as pd
#######################################################################

from lib.helpers import * #the helper provided for the project

# choose which implementations you would like
from lib.implementations import *
from lib.costs import *
from lib.preprocessing_functions import *

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [14]:
DATA_FOLDER = 'data/'

y_train, tx_train, ids_train = load_csv_data(DATA_FOLDER+'train.csv',sub_sample=False)

# TODOOOOOOOOOO GET RID OF IT BEFORE SUBMITTING CODE!
size_train = int(3*len(y_train)/4)
y_test = y_train[:size_train]
tx_test = tx_train[:size_train,:]
ids_test = ids_train[:size_train]

y_train = y_train[size_train:]
tx_train = tx_train[size_train:,:]
ids_train = ids_train[size_train:]

#y_test, tx_test, ids_test = load_csv_data(DATA_FOLDER+'test.csv',sub_sample=False)

In [15]:
print(tx_train.shape)
print(tx_test.shape)
print(y_train.shape)

(62500, 30)
(187500, 30)
(62500,)


In [16]:
# illegal: 
pd.DataFrame(tx_train).head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,96.301,2.125,69.796,16.756,-999.0,-999.0,-999.0,2.890,16.756,67.376,...,1.194,171.637,0.0,-999.000,-999.000,-999.000,-999.0,-999.0,-999.0,0.000
1,107.314,86.968,73.504,3.357,-999.0,-999.0,-999.0,2.654,3.357,81.061,...,-2.001,84.373,0.0,-999.000,-999.000,-999.000,-999.0,-999.0,-999.0,0.000
2,92.724,53.581,67.662,2.382,-999.0,-999.0,-999.0,2.984,2.382,65.424,...,1.200,70.184,0.0,-999.000,-999.000,-999.000,-999.0,-999.0,-999.0,0.000
3,-999.000,88.819,92.609,48.704,-999.0,-999.0,-999.0,2.366,36.973,141.492,...,-0.890,244.647,1.0,64.359,0.887,-2.142,-999.0,-999.0,-999.0,64.359
4,124.788,32.193,79.590,28.443,-999.0,-999.0,-999.0,3.209,28.443,61.223,...,-2.818,130.209,0.0,-999.000,-999.000,-999.000,-999.0,-999.0,-999.0,0.000


# Methods

### Functions for methods: 

OUT_FOLDER = 'output/'

def func_least_squares (y, tx, test_set, fct='mse'):
    #name = 'least_squares'
    w,loss = least_squares(y,tx,fct)
    #y_pred = predict_labels(w, test_set)
    #create_csv_submission(ids_test, y_pred, OUT_FOLDER+name)
    
    plt.plot(w, 'go')
    plt.title('least squares weights for loss:  '+str(loss),fontsize=15, fontweight='bold');
    plt.show()
    
    return w, loss

def func_GD (y, tx, test_set, max_iters, gamma, initial_w):
    w,loss = least_squares_GD(y, tx, initial_w, max_iters, gamma,fct='mse');
    #y_pred = predict_labels(w, test_set)
    #create_csv_submission(ids_test, y_pred, OUT_FOLDER+name)
    
    plt.plot(w, 'go')
    plt.title('GD: weights with loss:  '+str(loss),fontsize=15, fontweight='bold');
    plt.show()
    
    return w, loss

########################  RIDGE REGRESSION #######################################################
def func_ridge_regression (y, tx, test_set, lambda_):
    name = 'Ridge_regression'
    
    w,loss = ridge_regression(y, tx, lambda_=lambda_, fct='mse');
    
    y_pred = predict_labels(w, test_set)
    create_csv_submission(ids_test, y_pred, OUT_FOLDER+name)
    
    plt.plot(w, 'go')
    plt.title('Ridge regression: weights ;loss:  '+str(loss),fontsize=15, fontweight='bold');
    plt.show()
    
    return w, loss

In [17]:
def build_k_indices(y, k_fold, seed):
    """Build k indices for k-fold."""
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval]
                 for k in range(k_fold)]
    return np.array(k_indices)

## LEAST-SQUARES

In [18]:
def cross_validation_one_fold_LS(y_cross_val_train, y_cross_val_test, tx_cross_val_train, tx_cross_val_test, \
                                 degrees, len_kept_data, stdize=False):
    
    accuracies_train_by_deg = np.zeros(len(degrees))
    accuracies_test_by_deg = np.zeros(len(degrees))
    
    # For each degree, compute the least squares weights, the predictions and the accuracies
    previous_deg = 1
    for deg_id, deg in enumerate(degrees):
        print('++ Degree', deg, '++')
        
        # Add powers of the chosen columns
        len_data = tx_cross_val_train.shape[1]
        tx_cross_val_train = add_powers(tx_cross_val_train, range(previous_deg+1, deg+1), range(len_kept_data))
        tx_cross_val_test = add_powers(tx_cross_val_test, range(previous_deg+1, deg+1), range(len_kept_data))
        if stdize: 
            tx_cross_val_train[:,len_data:] = standardize(tx_cross_val_train[:,len_data:])[0]
            tx_cross_val_test[:,len_data:] = standardize(tx_cross_val_test[:,len_data:])[0]
    
        # Compute the best weights on the training set
        weights, loss = least_squares(y_cross_val_train, tx_cross_val_train, 'mse') 

        # Compute the predictions
        y_predicted_cross_val_train = predict_labels(weights, tx_cross_val_train)
        y_predicted_cross_val_test = predict_labels(weights, tx_cross_val_test)

        # Compute the accuracies for each degree
        accuracies_train_by_deg[deg_id] = \
            np.sum(y_predicted_cross_val_train == y_cross_val_train)/len(y_cross_val_train)
        accuracies_test_by_deg[deg_id] = \
            np.sum(y_predicted_cross_val_test == y_cross_val_test)/len(y_cross_val_test)
        
        # Update the previous degree to the actual degree
        previous_deg = deg
        
    return accuracies_train_by_deg, accuracies_test_by_deg

In [19]:
def cross_validation_least_squares(y_single_jet_train, tx_single_jet_train, degrees, k_fold, seed):
    
    # Get the indices so that we get the k'th subgroup in test, others in train, for each k
    k_indices = build_k_indices(y_single_jet_train, k_fold, seed)
    
    # Initialize matrix of computed accuracies for each degree and each fold
    accuracies_train_by_fold = np.zeros([len(degrees), k_fold])
    accuracies_test_by_fold = np.zeros([len(degrees), k_fold])
    
    # Preprocess training dataset
    tx_single_jet_train_preprocessed, len_kept_data, unique_cols = \
        preprocess_data(tx_single_jet_train, [], 'none')
    
    for k in range(k_fold):
        print('--- Fold', k, '---')
        # Create the testing set for this fold number
        k_index = k_indices[k] # Indices of the testing set for fold k
        y_cross_val_test = y_single_jet_train[k_index]
        tx_cross_val_test = tx_single_jet_train_preprocessed[k_index,:]
        
        # Create the training set for this fold number
        mask = np.ones(len(y_single_jet_train), dtype=bool) # set all elements to True
        mask[k_index] = False # set test elements to False
        y_cross_val_train = y_single_jet_train[mask] # select only True elements (ie train elements)
        tx_cross_val_train = tx_single_jet_train_preprocessed[mask,:]
        
        # Compute the accuracies for each degree
        accuracies_train_by_fold[:,k], accuracies_test_by_fold[:,k] = cross_validation_one_fold_LS\
            (y_cross_val_train, y_cross_val_test, tx_cross_val_train, tx_cross_val_test, degrees, len_kept_data, \
             False)
    
    # Compute the mean accuracies over the folds, for each degree
    mean_accuracies_train_by_deg = np.mean(accuracies_train_by_fold, axis=1)
    mean_accuracies_test_by_deg = np.mean(accuracies_test_by_fold, axis=1)
    
    # Get the index of the best accuracy in the testing set
    max_id_deg_test = np.unravel_index(mean_accuracies_test_by_deg.argmax(), mean_accuracies_test_by_deg.shape)
    
    # Find the optimal degree and the corresponding accuracies in the training and testing sets
    best_deg = degrees[max_id_deg_test[0]]
    best_accuracy_test = mean_accuracies_test_by_deg[max_id_deg_test[0]]
    corresponding_accuracy_train = mean_accuracies_train_by_deg[max_id_deg_test[0]]
    
    print('Best accuracy test =', best_accuracy_test, 'with degree =', best_deg)
    print('Corresponding accuracy train =', corresponding_accuracy_train)
    
    return best_deg, best_accuracy_test, corresponding_accuracy_train

In [20]:
degrees = range(6,11)
k_fold = 5
seed = 1

In [21]:
mask_jets_train = split_jets_mask(tx_train)
mask_jets_test = split_jets_mask(tx_test)
len_mask = len(mask_jets_train)

y_predicted_train = np.zeros(len(y_train))
y_predicted_test = np.zeros(tx_test.shape[0])
best_degrees = np.zeros(len_mask)

In [22]:
for jet_id in range(len_mask):
    print('***** Jet ', jet_id, '*****')
    # SEPARATE THE WHOLE DATA SET TO GET ONLY THE PART THAT HAVE THE RIGHT NUMBER OF JETS
    tx_single_jet_train = tx_train[mask_jets_train[jet_id]]
    tx_single_jet_test = tx_test[mask_jets_test[jet_id]]
    y_single_jet_train = y_train[mask_jets_train[jet_id]]
    
    # CALL CROSS VALIDATION FOR A SINGLE JET ON TRAIN PART, FIND BEST DEG, BEST ACCURACY ON TESTING CROSS VAL
    best_deg, best_accuracy, corresponding_accuracy_train = cross_validation_least_squares(y_single_jet_train, \
                                                                                           tx_single_jet_train, \
                                                                                           degrees, k_fold, seed)
    
    # KEEP IN MEMORY THE BEST DEGREE FOR THIS JET
    best_degrees[jet_id] = best_deg
    
    # PREPROCESS FULL TRAINING AND TESTING DATA
    tx_single_jet_train_preprocessed, len_kept_data, unique_cols = \
        preprocess_data(tx_single_jet_train, [], 'none')
    tx_single_jet_test_preprocessed = preprocess_data(tx_single_jet_test, unique_cols, 'none')[0]
    
    # ADD POWERS TO THE CHOSEN COLUMNS
    tx_single_jet_train_preprocessed = add_powers(tx_single_jet_train_preprocessed, range(2,best_deg+1), \
                                                  range(len_kept_data))
    tx_single_jet_test_preprocessed = add_powers(tx_single_jet_test_preprocessed, range(2,best_deg+1), \
                                                 range(len_kept_data))
    
    # COMPUTE THE BEST WEIGHTS AND FULL ACCURACY ON TRAINING FULL SET - ONE JET
    weights, loss = least_squares(y_single_jet_train, tx_single_jet_train_preprocessed, 'mse')
    
    # COMPUTE THE PREDICTIONS ON THE FULL TESTING SET - SINGLE JET
    y_predicted_single_jet_train = predict_labels(weights, tx_single_jet_train_preprocessed)
    y_predicted_single_jet_test = predict_labels(weights, tx_single_jet_test_preprocessed)
    
    # ADD THE PREDICTIONS TO y_predicted_test AND y_predicted_train
    y_predicted_train[mask_jets_train[jet_id]] = y_predicted_single_jet_train
    y_predicted_test[mask_jets_test[jet_id]] = y_predicted_single_jet_test
    
    # COMPUTE THE ACCURACY train ON JET
    accuracy_train_single_jet = np.sum(y_predicted_single_jet_train == y_single_jet_train)/len(y_single_jet_train)
    
    # PRINT ACCURACY train ON JET
    print('Accuracy full train on jet', jet_id, '=', accuracy_train_single_jet)

***** Jet  0 *****
--- Fold 0 ---
++ Degree 6 ++


/home/zampieri/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
/home/zampieri/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


++ Degree 7 ++
++ Degree 8 ++
++ Degree 9 ++
++ Degree 10 ++
--- Fold 1 ---
++ Degree 6 ++
++ Degree 7 ++
++ Degree 8 ++
++ Degree 9 ++
++ Degree 10 ++
--- Fold 2 ---
++ Degree 6 ++
++ Degree 7 ++
++ Degree 8 ++
++ Degree 9 ++
++ Degree 10 ++
--- Fold 3 ---
++ Degree 6 ++
++ Degree 7 ++
++ Degree 8 ++
++ Degree 9 ++
++ Degree 10 ++
--- Fold 4 ---
++ Degree 6 ++
++ Degree 7 ++
++ Degree 8 ++
++ Degree 9 ++
++ Degree 10 ++
Best accuracy test = 0.0 with degree = 6
Corresponding accuracy train = 0.0


TypeError: NumPy boolean array indexing assignment requires a 0 or 1-dimensional input, input has 2 dimensions

In [ ]:
# CREATE CSV SUBMISSION
#create_csv_submission(ids_test, y_predicted_test, 'output/trial.csv')

# COMPUTE ACCURACY ON FULL train
total_accuracy_train = np.sum(y_predicted_train == y_train)/len(y_train)*100
print('Total accuracy train =', total_accuracy_train, 'with degrees =', best_degrees)

In [ ]:
total_accuracy_test = np.sum(y_predicted_test == y_test)/len(y_test)*100
print('Total accuracy test =', total_accuracy_test, 'with degrees =', best_degrees)

In [ ]:
np.sum(y_predicted_test==-1)

In [ ]:
np.sum(y_predicted_test==1)

In [ ]:
y_predicted_test[:200]

## Gradient descent 

In [ ]:
def cross_validation_one_fold_GD(y_cross_val_train, y_cross_val_test, tx_cross_val_train, tx_cross_val_test, \
                                 degrees, gammas, len_kept_data, max_iters, stdize=False):
    
    accuracies_train_by_deg = np.zeros([len(degrees), len(gammas)])
    accuracies_test_by_deg = np.zeros([len(degrees), len(gammas)])
    
    # For each degree, compute the least squares weights, the predictions and the accuracies
    previous_deg = 1
    for deg_id, deg in enumerate(degrees):
        print('++ Degree', deg, '++')
                
        # Add powers of the chosen columns
        len_data = tx_cross_val_train.shape[1]
        tx_cross_val_train = add_powers(tx_cross_val_train, range(previous_deg+1, deg+1), range(len_kept_data))
        tx_cross_val_test = add_powers(tx_cross_val_test, range(previous_deg+1, deg+1), range(len_kept_data))
        if stdize: 
            tx_cross_val_train[:,len_data:] = standardize(tx_cross_val_train[:,len_data:])[0]
            tx_cross_val_test[:,len_data:] = standardize(tx_cross_val_test[:,len_data:])[0]
                
        for gamma_id, single_gamma in enumerate(gammas):
            print('>> Gamma', single_gamma, '<<')
            
            # Compute the best weights on the training set
            initial_w = np.zeros(tx_cross_val_train.shape[1])
            weights, loss = least_squares_GD(y_cross_val_train, tx_cross_val_train, initial_w, max_iters, \
                                             single_gamma, fct='mse');

            # Compute the predictions
            y_predicted_cross_val_train = predict_labels(weights, tx_cross_val_train)
            y_predicted_cross_val_test = predict_labels(weights, tx_cross_val_test)

            # Compute the accuracies for each degree
            accuracies_train_by_deg[deg_id, gamma_id] = \
                np.sum(y_predicted_cross_val_train == y_cross_val_train)/len(y_cross_val_train)
            accuracies_test_by_deg[deg_id, gamma_id] = \
                np.sum(y_predicted_cross_val_test == y_cross_val_test)/len(y_cross_val_test)

        # Update the previous degree to the actual degree
        previous_deg = deg
        
    return accuracies_train_by_deg, accuracies_test_by_deg

In [ ]:
def cross_validation_GD(y_single_jet_train, tx_single_jet_train, degrees, gammas, k_fold, seed, max_iters):
    
    # Get the indices so that we get the k'th subgroup in test, others in train, for each k
    k_indices = build_k_indices(y_single_jet_train, k_fold, seed)
    
    # Initialize matrix of computed accuracies for each degree and each fold
    accuracies_train_by_fold = np.zeros([len(degrees), len(gammas), k_fold])
    accuracies_test_by_fold = np.zeros([len(degrees), len(gammas), k_fold])
    
    # Preprocess training dataset
    tx_single_jet_train_preprocessed, len_kept_data, unique_cols = \
        preprocess_data(tx_single_jet_train, [], 'after')
    
    for k in range(k_fold):
        print('--- Fold', k, '---')
        # Create the testing set for this fold number
        k_index = k_indices[k] # Indices of the testing set for fold k
        y_cross_val_test = y_single_jet_train[k_index]
        tx_cross_val_test = tx_single_jet_train_preprocessed[k_index,:]
        
        # Create the training set for this fold number
        mask = np.ones(len(y_single_jet_train), dtype=bool) # set all elements to True
        mask[k_index] = False # set test elements to False
        y_cross_val_train = y_single_jet_train[mask] # select only True elements (ie train elements)
        tx_cross_val_train = tx_single_jet_train_preprocessed[mask,:]
        
        # Compute the accuracies for each degree
        accuracies_train_by_fold[:,:,k], accuracies_test_by_fold[:,:,k] = cross_validation_one_fold_GD\
            (y_cross_val_train, y_cross_val_test, tx_cross_val_train, tx_cross_val_test, \
                                 degrees, gammas, len_kept_data, max_iters, True)
    
    # Compute the mean accuracies over the folds, for each degree
    mean_accuracies_train_by_deg = np.mean(accuracies_train_by_fold, axis=2)
    mean_accuracies_test_by_deg = np.mean(accuracies_test_by_fold, axis=2)
    
    # Get the index of the best accuracy in the testing set
    max_id_deg_test, max_id_gamma_test = \
        np.unravel_index(mean_accuracies_test_by_deg.argmax(), mean_accuracies_test_by_deg.shape)
    
    # Find the optimal degree and the corresponding accuracies in the training and testing sets
    best_deg = degrees[max_id_deg_test]
    best_gamma = gammas[max_id_gamma_test]
    best_accuracy_test = mean_accuracies_test_by_deg[max_id_deg_test, max_id_gamma_test]
    corresponding_accuracy_train = mean_accuracies_train_by_deg[max_id_deg_test, max_id_gamma_test]
    
    print('Best accuracy test =', best_accuracy_test, 'with degree =', best_deg, 'and gamma =', best_gamma)
    print('Corresponding accuracy train =', corresponding_accuracy_train)
    
    return best_deg, best_gamma, best_accuracy_test, corresponding_accuracy_train

In [ ]:
degrees = range(6,11)
gammas = np.logspace(-5,-1,5)
k_fold = 5
seed = 1
max_iters = 300

In [ ]:
mask_jets_train = split_jets_mask(tx_train)
mask_jets_test = split_jets_mask(tx_test)
len_mask = len(mask_jets_train)

y_predicted_train = np.zeros(len(y_train))
y_predicted_test = np.zeros(tx_test.shape[0])
best_degrees = np.zeros(len_mask)
best_gammas = np.zeros(len_mask)

In [ ]:
for jet_id in range(len_mask):
    print('***** Jet ', jet_id, '*****')
    # SEPARATE THE WHOLE DATA SET TO GET ONLY THE PART THAT HAVE THE RIGHT NUMBER OF JETS
    tx_single_jet_train = tx_train[mask_jets_train[jet_id]]
    tx_single_jet_test = tx_test[mask_jets_test[jet_id]]
    y_single_jet_train = y_train[mask_jets_train[jet_id]]
    
    # CALL CROSS VALIDATION FOR A SINGLE JET ON TRAIN PART, FIND BEST DEG, BEST ACCURACY ON TESTING CROSS VAL
    best_deg, best_gamma, best_accuracy_test, corresponding_accuracy_train = \
        cross_validation_GD(y_single_jet_train, tx_single_jet_train, degrees, gammas, k_fold, seed, max_iters)
    
    # KEEP IN MEMORY THE BEST DEGREE FOR THIS JET
    best_degrees[jet_id] = best_deg
    best_gammas[jet_id] = best_gamma
    
    # PREPROCESS FULL TRAINING AND TESTING DATA
    tx_single_jet_train_preprocessed, len_kept_data, unique_cols = \
        preprocess_data(tx_single_jet_train, [], 'after')
    tx_single_jet_test_preprocessed = preprocess_data(tx_single_jet_test, unique_cols, 'after')[0]
    
    # ADD POWERS TO THE CHOSEN COLUMNS
    len_data = tx_single_jet_train_preprocessed.shape[1]
    tx_single_jet_train_preprocessed = add_powers(tx_single_jet_train_preprocessed, range(2,best_deg+1), \
                                                  range(len_kept_data))
    tx_single_jet_test_preprocessed = add_powers(tx_single_jet_test_preprocessed, range(2,best_deg+1), \
                                                 range(len_kept_data))
    tx_single_jet_train_preprocessed[:,len_data:] = standardize(tx_single_jet_train_preprocessed[:,len_data:])[0]
    tx_single_jet_test_preprocessed[:,len_data:] = standardize(tx_single_jet_test_preprocessed[:,len_data:])[0]
    
    # COMPUTE THE BEST WEIGHTS AND FULL ACCURACY ON TRAINING FULL SET - ONE JET
    initial_w = np.zeros(tx_single_jet_train_preprocessed.shape[1])
    weights, loss = least_squares_GD(y_single_jet_train, tx_single_jet_train_preprocessed, initial_w, max_iters, \
                                             best_gamma);
    
    # COMPUTE THE PREDICTIONS ON THE FULL TESTING SET - SINGLE JET
    y_predicted_single_jet_train = predict_labels(weights, tx_single_jet_train_preprocessed)
    y_predicted_single_jet_test = predict_labels(weights, tx_single_jet_test_preprocessed)
    
    # ADD THE PREDICTIONS TO y_predicted_test AND y_predicted_train
    y_predicted_train[mask_jets_train[jet_id]] = y_predicted_single_jet_train
    y_predicted_test[mask_jets_test[jet_id]] = y_predicted_single_jet_test
    
    # COMPUTE THE ACCURACY train ON JET
    accuracy_train_single_jet = np.sum(y_predicted_single_jet_train == y_single_jet_train)/len(y_single_jet_train)
    
    # PRINT ACCURACY train ON JET
    print('Accuracy full train on jet', jet_id, '=', accuracy_train_single_jet)

In [ ]:
# CREATE CSV SUBMISSION
#create_csv_submission(ids_test, y_predicted_test, 'output/trial.csv')

# COMPUTE ACCURACY ON FULL train
total_accuracy_train = np.sum(y_predicted_train == y_train)/len(y_train)*100
print('Total accuracy train =', total_accuracy_train, 'with degrees =', best_degrees, 'and gammas =', best_gammas)

In [22]:
total_accuracy_test = np.sum(y_predicted_test == y_test)/len(y_test)*100
print('Total accuracy test =', total_accuracy_test, 'with degrees =', best_degrees)

Total accuracy test = 79.2613333333 with degrees = [  9.  10.   8.   9.]


In [23]:
np.sum(y_predicted_test==-1)

133212

In [24]:
np.sum(y_predicted_test==1)

54288

In [25]:
y_predicted_test[:200]

array([-1., -1., -1., -1., -1., -1.,  1.,  1., -1.,  1., -1., -1.,  1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,  1., -1., -1.,
        1.,  1.,  1.,  1., -1.,  1.,  1.,  1., -1., -1.,  1., -1., -1.,
       -1., -1., -1.,  1., -1., -1., -1.,  1., -1.,  1.,  1., -1.,  1.,
       -1., -1., -1., -1., -1., -1.,  1., -1.,  1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1.,  1., -1.,  1., -1., -1.,
        1.,  1., -1., -1., -1.,  1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1.,  1.,  1., -1., -1., -1., -1., -1., -1., -1., -1.,  1.,
       -1., -1.,  1.,  1., -1., -1.,  1.,  1., -1., -1., -1.,  1., -1.,
       -1., -1.,  1., -1., -1., -1., -1., -1.,  1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1.,  1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1.,  1.,  1.,  1.,  1., -1., -1., -1.,  1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,  1.,  1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1

## Ridge regression   

In [26]:
def cross_validation_one_fold_ridge(y_cross_val_train, y_cross_val_test, tx_cross_val_train, tx_cross_val_test, \
                                    degrees, lambdas, len_kept_data, stdize=False):
    
    accuracies_train_by_deg = np.zeros([len(degrees), len(lambdas)])
    accuracies_test_by_deg = np.zeros([len(degrees), len(lambdas)])
    
    # For each degree, compute the least squares weights, the predictions and the accuracies
    previous_deg = 1
    for deg_id, deg in enumerate(degrees):
        print('++ Degree', deg, '++')
                
        # Add powers of the chosen columns
        len_data = tx_cross_val_train.shape[1]
        tx_cross_val_train = add_powers(tx_cross_val_train, range(previous_deg+1, deg+1), range(len_kept_data))
        tx_cross_val_test = add_powers(tx_cross_val_test, range(previous_deg+1, deg+1), range(len_kept_data))
        if stdize: 
            tx_cross_val_train[:,len_data:] = standardize(tx_cross_val_train[:,len_data:])[0]
            tx_cross_val_test[:,len_data:] = standardize(tx_cross_val_test[:,len_data:])[0]
                
        for lambda_id, single_lambda in enumerate(lambdas):
            print('>> Lambda', single_lambda, '<<')
            
            # Compute the best weights on the training set
            weights, loss = ridge_regression(y_cross_val_train, tx_cross_val_train, single_lambda, 'mse');

            # Compute the predictions
            y_predicted_cross_val_train = predict_labels(weights, tx_cross_val_train)
            y_predicted_cross_val_test = predict_labels(weights, tx_cross_val_test)

            # Compute the accuracies for each degree
            accuracies_train_by_deg[deg_id, lambda_id] = \
                np.sum(y_predicted_cross_val_train == y_cross_val_train)/len(y_cross_val_train)
            accuracies_test_by_deg[deg_id, lambda_id] = \
                np.sum(y_predicted_cross_val_test == y_cross_val_test)/len(y_cross_val_test)

        # Update the previous degree to the actual degree
        previous_deg = deg
        
    return accuracies_train_by_deg, accuracies_test_by_deg

In [27]:
def cross_validation_ridge(y_single_jet_train, tx_single_jet_train, degrees, lambdas, k_fold, seed):
    
    # Get the indices so that we get the k'th subgroup in test, others in train, for each k
    k_indices = build_k_indices(y_single_jet_train, k_fold, seed)
    
    # Initialize matrix of computed accuracies for each degree and each fold
    accuracies_train_by_fold = np.zeros([len(degrees), len(lambdas), k_fold])
    accuracies_test_by_fold = np.zeros([len(degrees), len(lambdas), k_fold])
    
    # Preprocess training dataset
    tx_single_jet_train_preprocessed, len_kept_data, unique_cols = \
        preprocess_data(tx_single_jet_train, [], 'before')
    
    for k in range(k_fold):
        print('--- Fold', k, '---')
        # Create the testing set for this fold number
        k_index = k_indices[k] # Indices of the testing set for fold k
        y_cross_val_test = y_single_jet_train[k_index]
        tx_cross_val_test = tx_single_jet_train_preprocessed[k_index,:]
        
        # Create the training set for this fold number
        mask = np.ones(len(y_single_jet_train), dtype=bool) # set all elements to True
        mask[k_index] = False # set test elements to False
        y_cross_val_train = y_single_jet_train[mask] # select only True elements (ie train elements)
        tx_cross_val_train = tx_single_jet_train_preprocessed[mask,:]
        
        # Compute the accuracies for each degree
        accuracies_train_by_fold[:,:,k], accuracies_test_by_fold[:,:,k] = \
            cross_validation_one_fold_ridge(y_cross_val_train, y_cross_val_test, tx_cross_val_train, \
                                            tx_cross_val_test, degrees, lambdas, len_kept_data, False)
    
    # Compute the mean accuracies over the folds, for each degree
    mean_accuracies_train_by_deg = np.mean(accuracies_train_by_fold, axis=2)
    mean_accuracies_test_by_deg = np.mean(accuracies_test_by_fold, axis=2)
    
    # Get the index of the best accuracy in the testing set
    max_id_deg_test, max_id_lambda_test = \
        np.unravel_index(mean_accuracies_test_by_deg.argmax(), mean_accuracies_test_by_deg.shape)
    
    # Find the optimal degree and the corresponding accuracies in the training and testing sets
    best_deg = degrees[max_id_deg_test]
    best_lambda = lambdas[max_id_lambda_test]
    best_accuracy_test = mean_accuracies_test_by_deg[max_id_deg_test, max_id_lambda_test]
    corresponding_accuracy_train = mean_accuracies_train_by_deg[max_id_deg_test, max_id_lambda_test]
    
    print('Best accuracy test =', best_accuracy_test, 'with degree =', best_deg, 'and lambda =', best_lambda)
    print('Corresponding accuracy train =', corresponding_accuracy_train)
    
    return best_deg, best_lambda, best_accuracy_test, corresponding_accuracy_train

In [28]:
degrees = range(6,11)
lambdas = np.logspace(-9,-2,7)
k_fold = 5
seed = 1

In [29]:
mask_jets_train = split_jets_mask(tx_train)
mask_jets_test = split_jets_mask(tx_test)
len_mask = len(mask_jets_train)

y_predicted_train = np.zeros(len(y_train))
y_predicted_test = np.zeros(tx_test.shape[0])
best_degrees = np.zeros(len_mask)
best_lambdas = np.zeros(len_mask)

In [30]:
for jet_id in range(len_mask):
    print('***** Jet ', jet_id, '*****')
    # SEPARATE THE WHOLE DATA SET TO GET ONLY THE PART THAT HAVE THE RIGHT NUMBER OF JETS
    tx_single_jet_train = tx_train[mask_jets_train[jet_id]]
    tx_single_jet_test = tx_test[mask_jets_test[jet_id]]
    y_single_jet_train = y_train[mask_jets_train[jet_id]]
    
    # CALL CROSS VALIDATION FOR A SINGLE JET ON TRAIN PART, FIND BEST DEG, BEST ACCURACY ON TESTING CROSS VAL
    best_deg, best_lambda, best_accuracy_test, corresponding_accuracy_train = \
        cross_validation_ridge(y_single_jet_train, tx_single_jet_train, degrees, lambdas, k_fold, seed)
    
    # KEEP IN MEMORY THE BEST DEGREE FOR THIS JET
    best_degrees[jet_id] = best_deg
    best_lambdas[jet_id] = best_lambda
    
    # PREPROCESS FULL TRAINING AND TESTING DATA
    tx_single_jet_train_preprocessed, len_kept_data, unique_cols = \
        preprocess_data(tx_single_jet_train, [], 'before')
    tx_single_jet_test_preprocessed = preprocess_data(tx_single_jet_test, unique_cols, 'before')[0]
    
    # ADD POWERS TO THE CHOSEN COLUMNS
    len_data = tx_single_jet_train_preprocessed.shape[1]
    tx_single_jet_train_preprocessed = add_powers(tx_single_jet_train_preprocessed, range(2,best_deg+1), \
                                                  range(len_kept_data))
    tx_single_jet_test_preprocessed = add_powers(tx_single_jet_test_preprocessed, range(2,best_deg+1), \
                                                 range(len_kept_data))
    tx_single_jet_train_preprocessed[:,len_data:] = standardize(tx_single_jet_train_preprocessed[:,len_data:])[0]
    tx_single_jet_test_preprocessed[:,len_data:] = standardize(tx_single_jet_test_preprocessed[:,len_data:])[0]
    
    # COMPUTE THE BEST WEIGHTS AND FULL ACCURACY ON TRAINING FULL SET - ONE JET
    initial_w = np.zeros(tx_single_jet_train_preprocessed.shape[1])
    weights, loss = ridge_regression(y_single_jet_train, tx_single_jet_train_preprocessed, \
                                             best_lambda, fct='mse');
    
    # COMPUTE THE PREDICTIONS ON THE FULL TESTING SET - SINGLE JET
    y_predicted_single_jet_train = predict_labels(weights, tx_single_jet_train_preprocessed)
    y_predicted_single_jet_test = predict_labels(weights, tx_single_jet_test_preprocessed)
    
    # ADD THE PREDICTIONS TO y_predicted_test AND y_predicted_train
    y_predicted_train[mask_jets_train[jet_id]] = y_predicted_single_jet_train
    y_predicted_test[mask_jets_test[jet_id]] = y_predicted_single_jet_test
    
    # COMPUTE THE ACCURACY train ON JET
    accuracy_train_single_jet = np.sum(y_predicted_single_jet_train == y_single_jet_train)/len(y_single_jet_train)
    
    # PRINT ACCURACY train ON JET
    print('Accuracy full train on jet', jet_id, '=', accuracy_train_single_jet)

***** Jet  0 *****
--- Fold 0 ---
++ Degree 6 ++
>> Lambda 1e-09 <<
>> Lambda 1.46779926762e-08 <<
>> Lambda 2.15443469003e-07 <<
>> Lambda 3.16227766017e-06 <<
>> Lambda 4.64158883361e-05 <<
>> Lambda 0.000681292069058 <<
>> Lambda 0.01 <<
++ Degree 7 ++
>> Lambda 1e-09 <<
>> Lambda 1.46779926762e-08 <<
>> Lambda 2.15443469003e-07 <<
>> Lambda 3.16227766017e-06 <<
>> Lambda 4.64158883361e-05 <<
>> Lambda 0.000681292069058 <<
>> Lambda 0.01 <<
++ Degree 8 ++
>> Lambda 1e-09 <<
>> Lambda 1.46779926762e-08 <<
>> Lambda 2.15443469003e-07 <<
>> Lambda 3.16227766017e-06 <<
>> Lambda 4.64158883361e-05 <<
>> Lambda 0.000681292069058 <<
>> Lambda 0.01 <<
++ Degree 9 ++
>> Lambda 1e-09 <<
>> Lambda 1.46779926762e-08 <<
>> Lambda 2.15443469003e-07 <<
>> Lambda 3.16227766017e-06 <<
>> Lambda 4.64158883361e-05 <<
>> Lambda 0.000681292069058 <<
>> Lambda 0.01 <<
++ Degree 10 ++
>> Lambda 1e-09 <<
>> Lambda 1.46779926762e-08 <<
>> Lambda 2.15443469003e-07 <<
>> Lambda 3.16227766017e-06 <<
>> Lambda 

>> Lambda 1e-09 <<
>> Lambda 1.46779926762e-08 <<
>> Lambda 2.15443469003e-07 <<
>> Lambda 3.16227766017e-06 <<
>> Lambda 4.64158883361e-05 <<
>> Lambda 0.000681292069058 <<
>> Lambda 0.01 <<
++ Degree 10 ++
>> Lambda 1e-09 <<
>> Lambda 1.46779926762e-08 <<
>> Lambda 2.15443469003e-07 <<
>> Lambda 3.16227766017e-06 <<
>> Lambda 4.64158883361e-05 <<
>> Lambda 0.000681292069058 <<
>> Lambda 0.01 <<
--- Fold 3 ---
++ Degree 6 ++
>> Lambda 1e-09 <<
>> Lambda 1.46779926762e-08 <<
>> Lambda 2.15443469003e-07 <<
>> Lambda 3.16227766017e-06 <<
>> Lambda 4.64158883361e-05 <<
>> Lambda 0.000681292069058 <<
>> Lambda 0.01 <<
++ Degree 7 ++
>> Lambda 1e-09 <<
>> Lambda 1.46779926762e-08 <<
>> Lambda 2.15443469003e-07 <<
>> Lambda 3.16227766017e-06 <<
>> Lambda 4.64158883361e-05 <<
>> Lambda 0.000681292069058 <<
>> Lambda 0.01 <<
++ Degree 8 ++
>> Lambda 1e-09 <<
>> Lambda 1.46779926762e-08 <<
>> Lambda 2.15443469003e-07 <<
>> Lambda 3.16227766017e-06 <<
>> Lambda 4.64158883361e-05 <<
>> Lambda 0.0

>> Lambda 1e-09 <<
>> Lambda 1.46779926762e-08 <<
>> Lambda 2.15443469003e-07 <<
>> Lambda 3.16227766017e-06 <<
>> Lambda 4.64158883361e-05 <<
>> Lambda 0.000681292069058 <<
>> Lambda 0.01 <<
++ Degree 8 ++
>> Lambda 1e-09 <<
>> Lambda 1.46779926762e-08 <<
>> Lambda 2.15443469003e-07 <<
>> Lambda 3.16227766017e-06 <<
>> Lambda 4.64158883361e-05 <<
>> Lambda 0.000681292069058 <<
>> Lambda 0.01 <<
++ Degree 9 ++
>> Lambda 1e-09 <<
>> Lambda 1.46779926762e-08 <<
>> Lambda 2.15443469003e-07 <<
>> Lambda 3.16227766017e-06 <<
>> Lambda 4.64158883361e-05 <<
>> Lambda 0.000681292069058 <<
>> Lambda 0.01 <<
++ Degree 10 ++
>> Lambda 1e-09 <<
>> Lambda 1.46779926762e-08 <<
>> Lambda 2.15443469003e-07 <<
>> Lambda 3.16227766017e-06 <<
>> Lambda 4.64158883361e-05 <<
>> Lambda 0.000681292069058 <<
>> Lambda 0.01 <<
--- Fold 1 ---
++ Degree 6 ++
>> Lambda 1e-09 <<
>> Lambda 1.46779926762e-08 <<
>> Lambda 2.15443469003e-07 <<
>> Lambda 3.16227766017e-06 <<
>> Lambda 4.64158883361e-05 <<
>> Lambda 0.0

In [31]:
# CREATE CSV SUBMISSION
#create_csv_submission(ids_test, y_predicted_test, 'output/trial.csv')

# COMPUTE ACCURACY ON FULL train
total_accuracy_train = np.sum(y_predicted_train == y_train)/len(y_train)*100
print('Total accuracy train =', total_accuracy_train, 'with degrees =', best_degrees, 'and lambda =', best_lambdas)

Total accuracy train = 82.1136 with degrees = [ 10.  10.   9.  10.] and lambda = [  4.64158883e-05   1.00000000e-02   6.81292069e-04   1.00000000e-02]


In [32]:
total_accuracy_test = np.sum(y_predicted_test == y_test)/len(y_test)*100
print('Total accuracy test =', total_accuracy_test, 'with degrees =', best_degrees, 'and lambda =', best_lambdas)

Total accuracy test = 79.9717333333 with degrees = [ 10.  10.   9.  10.] and lambda = [  4.64158883e-05   1.00000000e-02   6.81292069e-04   1.00000000e-02]


In [33]:
np.sum(y_predicted_test==-1)

130484

In [34]:
np.sum(y_predicted_test==1)

57016

In [35]:
y_predicted_test[:200]

array([-1., -1., -1., -1., -1., -1.,  1.,  1., -1.,  1., -1., -1.,  1.,
       -1., -1., -1.,  1., -1., -1., -1., -1., -1., -1.,  1., -1., -1.,
       -1.,  1.,  1.,  1., -1., -1.,  1.,  1., -1., -1.,  1.,  1., -1.,
       -1., -1., -1.,  1., -1., -1., -1.,  1., -1.,  1.,  1., -1.,  1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1.,  1., -1.,  1., -1.,  1.,  1., -1.,
        1.,  1., -1., -1., -1.,  1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1.,  1.,  1., -1., -1., -1., -1., -1., -1., -1., -1.,  1.,
       -1., -1.,  1.,  1., -1., -1.,  1., -1.,  1., -1., -1.,  1., -1.,
       -1., -1.,  1.,  1., -1., -1., -1., -1.,  1., -1., -1.,  1., -1.,
       -1., -1., -1.,  1., -1.,  1., -1., -1., -1., -1., -1., -1.,  1.,
       -1.,  1., -1., -1.,  1.,  1.,  1.,  1., -1., -1., -1., -1., -1.,
       -1., -1.,  1., -1., -1., -1., -1., -1.,  1., -1.,  1.,  1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1

##  SGD

In [36]:
def cross_validation_one_fold_SGD(y_cross_val_train, y_cross_val_test, tx_cross_val_train, tx_cross_val_test, \
                                 degrees, gammas, len_kept_data, max_iters, batch_size, stdize=False):
    
    accuracies_train_by_deg = np.zeros([len(degrees), len(gammas)])
    accuracies_test_by_deg = np.zeros([len(degrees), len(gammas)])
    
    # For each degree, compute the least squares weights, the predictions and the accuracies
    previous_deg = 1
    for deg_id, deg in enumerate(degrees):
        print('++ Degree', deg, '++')
                
        # Add powers of the chosen columns
        len_data = tx_cross_val_train.shape[1]
        tx_cross_val_train = add_powers(tx_cross_val_train, range(previous_deg+1, deg+1), range(len_kept_data))
        tx_cross_val_test = add_powers(tx_cross_val_test, range(previous_deg+1, deg+1), range(len_kept_data))
        if stdize: 
            tx_cross_val_train[:,len_data:] = standardize(tx_cross_val_train[:,len_data:])[0]
            tx_cross_val_test[:,len_data:] = standardize(tx_cross_val_test[:,len_data:])[0]
                
        for gamma_id, single_gamma in enumerate(gammas):
            print('>> Gamma', single_gamma, '<<')
            
            # Compute the best weights on the training set
            initial_w = np.zeros(tx_cross_val_train.shape[1])
            weights, loss = least_squares_SGD(y_cross_val_train, tx_cross_val_train, initial_w, max_iters, \
                                             single_gamma, batch_size);

            # Compute the predictions
            y_predicted_cross_val_train = predict_labels(weights, tx_cross_val_train)
            y_predicted_cross_val_test = predict_labels(weights, tx_cross_val_test)

            # Compute the accuracies for each degree
            accuracies_train_by_deg[deg_id, gamma_id] = \
                np.sum(y_predicted_cross_val_train == y_cross_val_train)/len(y_cross_val_train)
            accuracies_test_by_deg[deg_id, gamma_id] = \
                np.sum(y_predicted_cross_val_test == y_cross_val_test)/len(y_cross_val_test)

        # Update the previous degree to the actual degree
        previous_deg = deg
        
    return accuracies_train_by_deg, accuracies_test_by_deg

In [37]:
def cross_validation_SGD(y_single_jet_train, tx_single_jet_train, degrees, gammas, k_fold, seed, max_iters, \
                        batch_size):
    
    # Get the indices so that we get the k'th subgroup in test, others in train, for each k
    k_indices = build_k_indices(y_single_jet_train, k_fold, seed)
    
    # Initialize matrix of computed accuracies for each degree and each fold
    accuracies_train_by_fold = np.zeros([len(degrees), len(gammas), k_fold])
    accuracies_test_by_fold = np.zeros([len(degrees), len(gammas), k_fold])
    
    # Preprocess training dataset
    tx_single_jet_train_preprocessed, len_kept_data, unique_cols = \
        preprocess_data(tx_single_jet_train, [], 'after')
    
    for k in range(k_fold):
        print('--- Fold', k, '---')
        # Create the testing set for this fold number
        k_index = k_indices[k] # Indices of the testing set for fold k
        y_cross_val_test = y_single_jet_train[k_index]
        tx_cross_val_test = tx_single_jet_train_preprocessed[k_index,:]
        
        # Create the training set for this fold number
        mask = np.ones(len(y_single_jet_train), dtype=bool) # set all elements to True
        mask[k_index] = False # set test elements to False
        y_cross_val_train = y_single_jet_train[mask] # select only True elements (ie train elements)
        tx_cross_val_train = tx_single_jet_train_preprocessed[mask,:]
        
        # Compute the accuracies for each degree
        accuracies_train_by_fold[:,:,k], accuracies_test_by_fold[:,:,k] = \
            cross_validation_one_fold_SGD(y_cross_val_train, y_cross_val_test, tx_cross_val_train, \
                                          tx_cross_val_test, degrees, gammas, len_kept_data, max_iters, \
                                          batch_size, True)
    
    # Compute the mean accuracies over the folds, for each degree
    mean_accuracies_train_by_deg = np.mean(accuracies_train_by_fold, axis=2)
    mean_accuracies_test_by_deg = np.mean(accuracies_test_by_fold, axis=2)
    
    # Get the index of the best accuracy in the testing set
    max_id_deg_test, max_id_gamma_test = \
        np.unravel_index(mean_accuracies_test_by_deg.argmax(), mean_accuracies_test_by_deg.shape)
    
    # Find the optimal degree and the corresponding accuracies in the training and testing sets
    best_deg = degrees[max_id_deg_test]
    best_gamma = gammas[max_id_gamma_test]
    best_accuracy_test = mean_accuracies_test_by_deg[max_id_deg_test, max_id_gamma_test]
    corresponding_accuracy_train = mean_accuracies_train_by_deg[max_id_deg_test, max_id_gamma_test]
    
    print('Best accuracy test =', best_accuracy_test, 'with degree =', best_deg, 'and gamma =', best_gamma)
    print('Corresponding accuracy train =', corresponding_accuracy_train)
    
    return best_deg, best_gamma, best_accuracy_test, corresponding_accuracy_train

In [38]:
degrees = range(3,10)
gammas = np.logspace(-6,-1,5)
k_fold = 5
seed = 1
max_iters = 300
batch_size = 1

In [39]:
mask_jets_train = split_jets_mask(tx_train)
mask_jets_test = split_jets_mask(tx_test)
len_mask = len(mask_jets_train)

y_predicted_train = np.zeros(len(y_train))
y_predicted_test = np.zeros(tx_test.shape[0])
best_degrees = np.zeros(len_mask)
best_gammas = np.zeros(len_mask)

In [40]:
for jet_id in range(len_mask):
    print('***** Jet ', jet_id, '*****')
    # SEPARATE THE WHOLE DATA SET TO GET ONLY THE PART THAT HAVE THE RIGHT NUMBER OF JETS
    tx_single_jet_train = tx_train[mask_jets_train[jet_id]]
    tx_single_jet_test = tx_test[mask_jets_test[jet_id]]
    y_single_jet_train = y_train[mask_jets_train[jet_id]]
    
    # CALL CROSS VALIDATION FOR A SINGLE JET ON TRAIN PART, FIND BEST DEG, BEST ACCURACY ON TESTING CROSS VAL
    best_deg, best_gamma, best_accuracy_test, corresponding_accuracy_train = \
        cross_validation_SGD(y_single_jet_train, tx_single_jet_train, degrees, gammas, k_fold, seed, max_iters, \
                            batch_size)
    
    # KEEP IN MEMORY THE BEST DEGREE FOR THIS JET
    best_degrees[jet_id] = best_deg
    best_gammas[jet_id] = best_gamma
    
    # PREPROCESS FULL TRAINING AND TESTING DATA
    tx_single_jet_train_preprocessed, len_kept_data, unique_cols = \
        preprocess_data(tx_single_jet_train, [], 'after')
    tx_single_jet_test_preprocessed = preprocess_data(tx_single_jet_test, unique_cols, 'after')[0]
    
    # ADD POWERS TO THE CHOSEN COLUMNS
    len_data = tx_single_jet_train_preprocessed.shape[1]
    tx_single_jet_train_preprocessed = add_powers(tx_single_jet_train_preprocessed, range(2,best_deg+1), \
                                                  range(len_kept_data))
    tx_single_jet_test_preprocessed = add_powers(tx_single_jet_test_preprocessed, range(2,best_deg+1), \
                                                 range(len_kept_data))
    tx_single_jet_train_preprocessed[:,len_data:] = standardize(tx_single_jet_train_preprocessed[:,len_data:])[0]
    tx_single_jet_test_preprocessed[:,len_data:] = standardize(tx_single_jet_test_preprocessed[:,len_data:])[0]
    
    # COMPUTE THE BEST WEIGHTS AND FULL ACCURACY ON TRAINING FULL SET - ONE JET
    initial_w = np.zeros(tx_single_jet_train_preprocessed.shape[1])
    weights, loss = least_squares_SGD(y_single_jet_train, tx_single_jet_train_preprocessed, initial_w, max_iters, \
                                             best_gamma, batch_size);
    
    # COMPUTE THE PREDICTIONS ON THE FULL TESTING SET - SINGLE JET
    y_predicted_single_jet_train = predict_labels(weights, tx_single_jet_train_preprocessed)
    y_predicted_single_jet_test = predict_labels(weights, tx_single_jet_test_preprocessed)
    
    # ADD THE PREDICTIONS TO y_predicted_test AND y_predicted_train
    y_predicted_train[mask_jets_train[jet_id]] = y_predicted_single_jet_train
    y_predicted_test[mask_jets_test[jet_id]] = y_predicted_single_jet_test
    
    # COMPUTE THE ACCURACY train ON JET
    accuracy_train_single_jet = np.sum(y_predicted_single_jet_train == y_single_jet_train)/len(y_single_jet_train)
    
    # PRINT ACCURACY train ON JET
    print('Accuracy full train on jet', jet_id, '=', accuracy_train_single_jet)

***** Jet  0 *****
--- Fold 0 ---
++ Degree 3 ++
>> Gamma 1e-06 <<
>> Gamma 1.77827941004e-05 <<
>> Gamma 0.000316227766017 <<
>> Gamma 0.0056234132519 <<
>> Gamma 0.1 <<
++ Degree 4 ++
>> Gamma 1e-06 <<
>> Gamma 1.77827941004e-05 <<
>> Gamma 0.000316227766017 <<
>> Gamma 0.0056234132519 <<
>> Gamma 0.1 <<
++ Degree 5 ++
>> Gamma 1e-06 <<
>> Gamma 1.77827941004e-05 <<
>> Gamma 0.000316227766017 <<
>> Gamma 0.0056234132519 <<
>> Gamma 0.1 <<
++ Degree 6 ++
>> Gamma 1e-06 <<
>> Gamma 1.77827941004e-05 <<
>> Gamma 0.000316227766017 <<
>> Gamma 0.0056234132519 <<
>> Gamma 0.1 <<
++ Degree 7 ++
>> Gamma 1e-06 <<
>> Gamma 1.77827941004e-05 <<
>> Gamma 0.000316227766017 <<
>> Gamma 0.0056234132519 <<
>> Gamma 0.1 <<
++ Degree 8 ++
>> Gamma 1e-06 <<
>> Gamma 1.77827941004e-05 <<
>> Gamma 0.000316227766017 <<
>> Gamma 0.0056234132519 <<
>> Gamma 0.1 <<
++ Degree 9 ++
>> Gamma 1e-06 <<
>> Gamma 1.77827941004e-05 <<
>> Gamma 0.000316227766017 <<
>> Gamma 0.0056234132519 <<
>> Gamma 0.1 <<
--- Fol

>> Gamma 0.000316227766017 <<
>> Gamma 0.0056234132519 <<
>> Gamma 0.1 <<
++ Degree 5 ++
>> Gamma 1e-06 <<
>> Gamma 1.77827941004e-05 <<
>> Gamma 0.000316227766017 <<
>> Gamma 0.0056234132519 <<
>> Gamma 0.1 <<
++ Degree 6 ++
>> Gamma 1e-06 <<
>> Gamma 1.77827941004e-05 <<
>> Gamma 0.000316227766017 <<
>> Gamma 0.0056234132519 <<
>> Gamma 0.1 <<
++ Degree 7 ++
>> Gamma 1e-06 <<
>> Gamma 1.77827941004e-05 <<
>> Gamma 0.000316227766017 <<
>> Gamma 0.0056234132519 <<
>> Gamma 0.1 <<
++ Degree 8 ++
>> Gamma 1e-06 <<
>> Gamma 1.77827941004e-05 <<
>> Gamma 0.000316227766017 <<
>> Gamma 0.0056234132519 <<
>> Gamma 0.1 <<
++ Degree 9 ++
>> Gamma 1e-06 <<
>> Gamma 1.77827941004e-05 <<
>> Gamma 0.000316227766017 <<
>> Gamma 0.0056234132519 <<
>> Gamma 0.1 <<
--- Fold 4 ---
++ Degree 3 ++
>> Gamma 1e-06 <<
>> Gamma 1.77827941004e-05 <<
>> Gamma 0.000316227766017 <<
>> Gamma 0.0056234132519 <<
>> Gamma 0.1 <<
++ Degree 4 ++
>> Gamma 1e-06 <<
>> Gamma 1.77827941004e-05 <<
>> Gamma 0.000316227766017

>> Gamma 0.0056234132519 <<
>> Gamma 0.1 <<
++ Degree 5 ++
>> Gamma 1e-06 <<
>> Gamma 1.77827941004e-05 <<
>> Gamma 0.000316227766017 <<
>> Gamma 0.0056234132519 <<
>> Gamma 0.1 <<
++ Degree 6 ++
>> Gamma 1e-06 <<
>> Gamma 1.77827941004e-05 <<
>> Gamma 0.000316227766017 <<
>> Gamma 0.0056234132519 <<
>> Gamma 0.1 <<
++ Degree 7 ++
>> Gamma 1e-06 <<
>> Gamma 1.77827941004e-05 <<
>> Gamma 0.000316227766017 <<
>> Gamma 0.0056234132519 <<
>> Gamma 0.1 <<
++ Degree 8 ++
>> Gamma 1e-06 <<
>> Gamma 1.77827941004e-05 <<
>> Gamma 0.000316227766017 <<
>> Gamma 0.0056234132519 <<
>> Gamma 0.1 <<
++ Degree 9 ++
>> Gamma 1e-06 <<
>> Gamma 1.77827941004e-05 <<
>> Gamma 0.000316227766017 <<
>> Gamma 0.0056234132519 <<
>> Gamma 0.1 <<
--- Fold 2 ---
++ Degree 3 ++
>> Gamma 1e-06 <<
>> Gamma 1.77827941004e-05 <<
>> Gamma 0.000316227766017 <<
>> Gamma 0.0056234132519 <<
>> Gamma 0.1 <<
++ Degree 4 ++
>> Gamma 1e-06 <<
>> Gamma 1.77827941004e-05 <<
>> Gamma 0.000316227766017 <<
>> Gamma 0.0056234132519 <

In [41]:
# CREATE CSV SUBMISSION
#create_csv_submission(ids_test, y_predicted_test, 'output/trial.csv')

# COMPUTE ACCURACY ON FULL train
total_accuracy_train = np.sum(y_predicted_train == y_train)/len(y_train)*100
print('Total accuracy train =', total_accuracy_train, 'with degrees =', best_degrees, 'and gammas =', best_gammas)

Total accuracy train = 67.7168 with degrees = [ 8.  9.  3.  4.] and gammas = [ 0.00031623  0.00031623  0.00031623  0.00031623]


In [42]:
total_accuracy_test = np.sum(y_predicted_test == y_test)/len(y_test)*100
print('Total accuracy test =', total_accuracy_test, 'with degrees =', best_degrees)

Total accuracy test = 67.8229333333 with degrees = [ 8.  9.  3.  4.]


In [43]:
np.sum(y_predicted_test==-1)

100295

In [44]:
np.sum(y_predicted_test==1)

87205

In [45]:
y_predicted_test[:200]

array([-1., -1., -1.,  1.,  1.,  1., -1.,  1., -1.,  1., -1.,  1.,  1.,
        1., -1.,  1., -1.,  1., -1., -1., -1.,  1., -1., -1., -1.,  1.,
        1.,  1.,  1.,  1., -1.,  1.,  1., -1.,  1.,  1.,  1.,  1., -1.,
       -1.,  1., -1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1., -1.,  1.,
       -1., -1., -1.,  1., -1., -1.,  1., -1.,  1., -1., -1.,  1.,  1.,
       -1.,  1., -1.,  1., -1., -1., -1., -1.,  1., -1.,  1., -1., -1.,
        1.,  1., -1., -1.,  1.,  1.,  1., -1., -1.,  1., -1.,  1., -1.,
        1.,  1.,  1.,  1., -1.,  1., -1., -1.,  1., -1., -1., -1.,  1.,
        1.,  1.,  1.,  1., -1., -1., -1.,  1., -1., -1., -1.,  1., -1.,
       -1., -1.,  1.,  1.,  1., -1., -1., -1.,  1.,  1.,  1., -1., -1.,
        1., -1., -1., -1., -1.,  1., -1.,  1., -1.,  1.,  1., -1., -1.,
        1.,  1., -1., -1.,  1., -1.,  1., -1., -1., -1., -1.,  1., -1.,
       -1., -1., -1.,  1., -1., -1., -1., -1., -1., -1.,  1.,  1.,  1.,
        1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,  1., -1

## LOGISTIC REGRESSION

In [46]:
def cross_validation_one_fold_logreg(y_cross_val_train, y_cross_val_test, tx_cross_val_train, tx_cross_val_test, \
                                    degrees, gammas, len_kept_data, max_iters, stdize=False):
    
    accuracies_train_by_deg = np.zeros([len(degrees), len(gammas)])
    accuracies_test_by_deg = np.zeros([len(degrees), len(gammas)])
    
    # For each degree, compute the least squares weights, the predictions and the accuracies
    previous_deg = 1
    for deg_id, deg in enumerate(degrees):
        print('++ Degree', deg, '++')
                
        # Add powers of the chosen columns
        len_data = tx_cross_val_train.shape[1]
        tx_cross_val_train = add_powers(tx_cross_val_train, range(previous_deg+1, deg+1), range(len_kept_data))
        tx_cross_val_test = add_powers(tx_cross_val_test, range(previous_deg+1, deg+1), range(len_kept_data))
        if stdize: 
            tx_cross_val_train[:,len_data:] = standardize(tx_cross_val_train[:,len_data:])[0]
            tx_cross_val_test[:,len_data:] = standardize(tx_cross_val_test[:,len_data:])[0]
                
        for gamma_id, single_gamma in enumerate(gammas):
            print('>> Gamma', single_gamma, '<<')
            
            # Compute the best weights on the training set
            initial_w = np.zeros(tx_cross_val_train.shape[1])
            weights, loss = logistic_regression(y_cross_val_train, tx_cross_val_train, initial_w, max_iters, \
                                                single_gamma);

            # Compute the predictions
            y_predicted_cross_val_train = predict_labels(weights, tx_cross_val_train)
            y_predicted_cross_val_test = predict_labels(weights, tx_cross_val_test)

            # Compute the accuracies for each degree
            accuracies_train_by_deg[deg_id, gamma_id] = \
                np.sum(y_predicted_cross_val_train == y_cross_val_train)/len(y_cross_val_train)
            accuracies_test_by_deg[deg_id, gamma_id] = \
                np.sum(y_predicted_cross_val_test == y_cross_val_test)/len(y_cross_val_test)

        # Update the previous degree to the actual degree
        previous_deg = deg
        
    return accuracies_train_by_deg, accuracies_test_by_deg

In [47]:
def cross_validation_logreg(y_single_jet_train, tx_single_jet_train, degrees, gammas, k_fold, seed, max_iters):
    
    # Get the indices so that we get the k'th subgroup in test, others in train, for each k
    k_indices = build_k_indices(y_single_jet_train, k_fold, seed)
    
    # Initialize matrix of computed accuracies for each degree and each fold
    accuracies_train_by_fold = np.zeros([len(degrees), len(gammas), k_fold])
    accuracies_test_by_fold = np.zeros([len(degrees), len(gammas), k_fold])
    
    # Preprocess training dataset
    tx_single_jet_train_preprocessed, len_kept_data, unique_cols = \
        preprocess_data(tx_single_jet_train, [], 'before')
    
    for k in range(k_fold):
        print('--- Fold', k, '---')
        # Create the testing set for this fold number
        k_index = k_indices[k] # Indices of the testing set for fold k
        y_cross_val_test = y_single_jet_train[k_index]
        tx_cross_val_test = tx_single_jet_train_preprocessed[k_index,:]
        
        # Create the training set for this fold number
        mask = np.ones(len(y_single_jet_train), dtype=bool) # set all elements to True
        mask[k_index] = False # set test elements to False
        y_cross_val_train = y_single_jet_train[mask] # select only True elements (ie train elements)
        tx_cross_val_train = tx_single_jet_train_preprocessed[mask,:]
        
        # Compute the accuracies for each degree
        accuracies_train_by_fold[:,:,k], accuracies_test_by_fold[:,:,k] = \
            cross_validation_one_fold_logreg(y_cross_val_train, y_cross_val_test, tx_cross_val_train, \
                                            tx_cross_val_test, degrees, gammas, len_kept_data, max_iters, False)
    
    # Compute the mean accuracies over the folds, for each degree
    mean_accuracies_train_by_deg = np.mean(accuracies_train_by_fold, axis=2)
    mean_accuracies_test_by_deg = np.mean(accuracies_test_by_fold, axis=2)
    
    # Get the index of the best accuracy in the testing set
    max_id_deg_test, max_id_gamma_test = \
        np.unravel_index(mean_accuracies_test_by_deg.argmax(), mean_accuracies_test_by_deg.shape)
    
    # Find the optimal degree and the corresponding accuracies in the training and testing sets
    best_deg = degrees[max_id_deg_test]
    best_gamma = gammas[max_id_gamma_test]
    best_accuracy_test = mean_accuracies_test_by_deg[max_id_deg_test, max_id_gamma_test]
    corresponding_accuracy_train = mean_accuracies_train_by_deg[max_id_deg_test, max_id_gamma_test]
    
    print('Best accuracy test =', best_accuracy_test, 'with degree =', best_deg, 'and gamma =', best_gamma)
    print('Corresponding accuracy train =', corresponding_accuracy_train)
    
    return best_deg, best_gamma, best_accuracy_test, corresponding_accuracy_train

In [48]:
degrees = range(6,10)
gammas = np.logspace(-9,-2,7)
k_fold = 5
seed = 1
max_iters = 300

In [49]:
mask_jets_train = split_jets_mask(tx_train)
mask_jets_test = split_jets_mask(tx_test)
len_mask = len(mask_jets_train)

y_predicted_train = np.zeros(len(y_train))
y_predicted_test = np.zeros(tx_test.shape[0])
best_degrees = np.zeros(len_mask)
best_lambdas = np.zeros(len_mask)

In [50]:
for jet_id in range(len_mask):
    print('***** Jet ', jet_id, '*****')
    # SEPARATE THE WHOLE DATA SET TO GET ONLY THE PART THAT HAVE THE RIGHT NUMBER OF JETS
    tx_single_jet_train = tx_train[mask_jets_train[jet_id]]
    tx_single_jet_test = tx_test[mask_jets_test[jet_id]]
    y_single_jet_train = y_train[mask_jets_train[jet_id]]
    
    # CALL CROSS VALIDATION FOR A SINGLE JET ON TRAIN PART, FIND BEST DEG, BEST ACCURACY ON TESTING CROSS VAL
    best_deg, best_gamma, best_accuracy_test, corresponding_accuracy_train = \
        cross_validation_logreg(y_single_jet_train, tx_single_jet_train, degrees, gammas, k_fold, seed, max_iters)
    
    # KEEP IN MEMORY THE BEST DEGREE FOR THIS JET
    best_degrees[jet_id] = best_deg
    best_gammas[jet_id] = best_gamma
    
    # PREPROCESS FULL TRAINING AND TESTING DATA
    tx_single_jet_train_preprocessed, len_kept_data, unique_cols = \
        preprocess_data(tx_single_jet_train, [], 'before')
    tx_single_jet_test_preprocessed = preprocess_data(tx_single_jet_test, unique_cols, 'before')[0]
    
    # ADD POWERS TO THE CHOSEN COLUMNS
    len_data = tx_single_jet_train_preprocessed.shape[1]
    tx_single_jet_train_preprocessed = add_powers(tx_single_jet_train_preprocessed, range(2,best_deg+1), \
                                                  range(len_kept_data))
    tx_single_jet_test_preprocessed = add_powers(tx_single_jet_test_preprocessed, range(2,best_deg+1), \
                                                 range(len_kept_data))
    tx_single_jet_train_preprocessed[:,len_data:] = standardize(tx_single_jet_train_preprocessed[:,len_data:])[0]
    tx_single_jet_test_preprocessed[:,len_data:] = standardize(tx_single_jet_test_preprocessed[:,len_data:])[0]
    
    # COMPUTE THE BEST WEIGHTS AND FULL ACCURACY ON TRAINING FULL SET - ONE JET
    initial_w = np.zeros(tx_single_jet_train_preprocessed.shape[1])
    weights, loss = logistic_regression(y_single_jet_train, tx_single_jet_train_preprocessed, initial_w,\
                                        max_iters, best_gamma);
    
    # COMPUTE THE PREDICTIONS ON THE FULL TESTING SET - SINGLE JET
    y_predicted_single_jet_train = predict_labels(weights, tx_single_jet_train_preprocessed)
    y_predicted_single_jet_test = predict_labels(weights, tx_single_jet_test_preprocessed)
    
    # ADD THE PREDICTIONS TO y_predicted_test AND y_predicted_train
    y_predicted_train[mask_jets_train[jet_id]] = y_predicted_single_jet_train
    y_predicted_test[mask_jets_test[jet_id]] = y_predicted_single_jet_test
    
    # COMPUTE THE ACCURACY train ON JET
    accuracy_train_single_jet = np.sum(y_predicted_single_jet_train == y_single_jet_train)/len(y_single_jet_train)
    
    # PRINT ACCURACY train ON JET
    print('Accuracy full train on jet', jet_id, '=', accuracy_train_single_jet)

***** Jet  0 *****
--- Fold 0 ---
++ Degree 6 ++
>> Gamma 1e-09 <<
>> Gamma 1.46779926762e-08 <<
>> Gamma 2.15443469003e-07 <<
>> Gamma 3.16227766017e-06 <<
>> Gamma 4.64158883361e-05 <<
>> Gamma 0.000681292069058 <<
>> Gamma 0.01 <<
++ Degree 7 ++
>> Gamma 1e-09 <<
>> Gamma 1.46779926762e-08 <<
>> Gamma 2.15443469003e-07 <<
>> Gamma 3.16227766017e-06 <<
>> Gamma 4.64158883361e-05 <<
>> Gamma 0.000681292069058 <<
>> Gamma 0.01 <<
++ Degree 8 ++
>> Gamma 1e-09 <<
>> Gamma 1.46779926762e-08 <<
>> Gamma 2.15443469003e-07 <<
>> Gamma 3.16227766017e-06 <<
>> Gamma 4.64158883361e-05 <<
>> Gamma 0.000681292069058 <<
>> Gamma 0.01 <<
++ Degree 9 ++
>> Gamma 1e-09 <<
>> Gamma 1.46779926762e-08 <<
>> Gamma 2.15443469003e-07 <<
>> Gamma 3.16227766017e-06 <<
>> Gamma 4.64158883361e-05 <<
>> Gamma 0.000681292069058 <<
>> Gamma 0.01 <<
--- Fold 1 ---
++ Degree 6 ++
>> Gamma 1e-09 <<
>> Gamma 1.46779926762e-08 <<
>> Gamma 2.15443469003e-07 <<
>> Gamma 3.16227766017e-06 <<
>> Gamma 4.64158883361e-05 <

>> Gamma 2.15443469003e-07 <<
>> Gamma 3.16227766017e-06 <<
>> Gamma 4.64158883361e-05 <<
>> Gamma 0.000681292069058 <<
>> Gamma 0.01 <<
Best accuracy test = 0.644868488912 with degree = 6 and gamma = 2.15443469003e-07
Corresponding accuracy train = 0.643638004383
Accuracy full train on jet 1 = 0.66692450495
***** Jet  2 *****
--- Fold 0 ---
++ Degree 6 ++
>> Gamma 1e-09 <<
>> Gamma 1.46779926762e-08 <<
>> Gamma 2.15443469003e-07 <<
>> Gamma 3.16227766017e-06 <<
>> Gamma 4.64158883361e-05 <<
>> Gamma 0.000681292069058 <<
>> Gamma 0.01 <<
++ Degree 7 ++
>> Gamma 1e-09 <<
>> Gamma 1.46779926762e-08 <<
>> Gamma 2.15443469003e-07 <<
>> Gamma 3.16227766017e-06 <<
>> Gamma 4.64158883361e-05 <<
>> Gamma 0.000681292069058 <<
>> Gamma 0.01 <<
++ Degree 8 ++
>> Gamma 1e-09 <<
>> Gamma 1.46779926762e-08 <<
>> Gamma 2.15443469003e-07 <<
>> Gamma 3.16227766017e-06 <<
>> Gamma 4.64158883361e-05 <<
>> Gamma 0.000681292069058 <<
>> Gamma 0.01 <<
++ Degree 9 ++
>> Gamma 1e-09 <<
>> Gamma 1.46779926762e

>> Gamma 4.64158883361e-05 <<
>> Gamma 0.000681292069058 <<
>> Gamma 0.01 <<
++ Degree 8 ++
>> Gamma 1e-09 <<
>> Gamma 1.46779926762e-08 <<
>> Gamma 2.15443469003e-07 <<
>> Gamma 3.16227766017e-06 <<
>> Gamma 4.64158883361e-05 <<
>> Gamma 0.000681292069058 <<
>> Gamma 0.01 <<
++ Degree 9 ++
>> Gamma 1e-09 <<
>> Gamma 1.46779926762e-08 <<
>> Gamma 2.15443469003e-07 <<
>> Gamma 3.16227766017e-06 <<
>> Gamma 4.64158883361e-05 <<
>> Gamma 0.000681292069058 <<
>> Gamma 0.01 <<
Best accuracy test = 0.710431654676 with degree = 8 and gamma = 1e-09
Corresponding accuracy train = 0.709238031018
Accuracy full train on jet 3 = 0.701672361086


In [51]:
# CREATE CSV SUBMISSION
#create_csv_submission(ids_test, y_predicted_test, 'output/trial.csv')

# COMPUTE ACCURACY ON FULL train
total_accuracy_train = np.sum(y_predicted_train == y_train)/len(y_train)*100
print('Total accuracy train =', total_accuracy_train, 'with degrees =', best_degrees, 'and gamma =', best_gammas)

Total accuracy train = 69.496 with degrees = [ 6.  6.  9.  8.] and gamma = [  1.00000000e-09   2.15443469e-07   1.46779927e-08   1.00000000e-09]


In [52]:
total_accuracy_test = np.sum(y_predicted_test == y_test)/len(y_test)*100
print('Total accuracy test =', total_accuracy_test, 'with degrees =', best_degrees, 'and gamma =', best_gammas)

Total accuracy test = 69.4848 with degrees = [ 6.  6.  9.  8.] and gamma = [  1.00000000e-09   2.15443469e-07   1.46779927e-08   1.00000000e-09]


In [53]:
np.sum(y_predicted_test==-1)

177881

In [54]:
np.sum(y_predicted_test==1)

9619

In [55]:
y_predicted_test[:200]

array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1.,  1., -1.,  1., -1., -1.,  1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1.,  1., -1., -1., -1., -1., -1., -1., -1., -1.,  1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1.,  1., -1., -1., -1., -1., -1.,  1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1.,  1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1.,  1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1.,  1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,  1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1

## LOGISTIC REGULARIZED

In [56]:
def cross_validation_one_fold_logistic_regularized(y_cross_val_train, y_cross_val_test, tx_cross_val_train, tx_cross_val_test, \
                                 degrees, gammas, lambdas, len_kept_data, max_iters, stdize=False):
    
    accuracies_train_by_deg = np.zeros([len(degrees), len(gammas),len(lambdas)])
    accuracies_test_by_deg = np.zeros([len(degrees), len(gammas),len(lambdas)])
    
    # For each degree, compute the least squares weights, the predictions and the accuracies
    previous_deg = 1
    for deg_id, deg in enumerate(degrees):
        print('++ Degree', deg, '++')
                
        # Add powers of the chosen columns
        len_data = tx_cross_val_train.shape[1]
        tx_cross_val_train = add_powers(tx_cross_val_train, range(previous_deg+1, deg+1), range(len_kept_data))
        tx_cross_val_test = add_powers(tx_cross_val_test, range(previous_deg+1, deg+1), range(len_kept_data))
        if stdize: 
            tx_cross_val_train[:,len_data:] = standardize(tx_cross_val_train[:,len_data:])[0]
            tx_cross_val_test[:,len_data:] = standardize(tx_cross_val_test[:,len_data:])[0]
                
        for gamma_id, single_gamma in enumerate(gammas):
            print('>> Gamma', single_gamma, '<<')
            
            
            for lambda_id, single_lambda in enumerate(lambdas):
                
                print('>> Lambda', single_lambda, '<<')
            # Compute the best weights on the training set
                initial_w = np.zeros(tx_cross_val_train.shape[1])
                weights, loss = reg_logistic_regression(y_cross_val_train, tx_cross_val_train, single_lambda,initial_w, max_iters, \
                                                 single_gamma);

                # Compute the predictions
                y_predicted_cross_val_train = predict_labels(weights, tx_cross_val_train)
                y_predicted_cross_val_test = predict_labels(weights, tx_cross_val_test)

                # Compute the accuracies for each degree
                accuracies_train_by_deg[deg_id, gamma_id,lambda_id] = \
                    np.sum(y_predicted_cross_val_train == y_cross_val_train)/len(y_cross_val_train)
                accuracies_test_by_deg[deg_id, gamma_id,lambda_id] = \
                    np.sum(y_predicted_cross_val_test == y_cross_val_test)/len(y_cross_val_test)

        # Update the previous degree to the actual degree
        previous_deg = deg
        
    return accuracies_train_by_deg, accuracies_test_by_deg

In [57]:
def cross_validation_logistic_regularized(y_single_jet_train, tx_single_jet_train, degrees, gammas, lambdas, k_fold, seed, max_iters):
    
    # Get the indices so that we get the k'th subgroup in test, others in train, for each k
    k_indices = build_k_indices(y_single_jet_train, k_fold, seed)
    
    # Initialize matrix of computed accuracies for each degree and each fold
    accuracies_train_by_fold = np.zeros([len(degrees), len(gammas),len(lambdas), k_fold])
    accuracies_test_by_fold = np.zeros([len(degrees), len(gammas),len(lambdas), k_fold])
    
    # Preprocess training dataset
    tx_single_jet_train_preprocessed, len_kept_data, unique_cols = \
        preprocess_data(tx_single_jet_train, [], 'after')
    
    for k in range(k_fold):
        print('--- Fold', k, '---')
        # Create the testing set for this fold number
        k_index = k_indices[k] # Indices of the testing set for fold k
        y_cross_val_test = y_single_jet_train[k_index]
        tx_cross_val_test = tx_single_jet_train_preprocessed[k_index,:]
        
        # Create the training set for this fold number
        mask = np.ones(len(y_single_jet_train), dtype=bool) # set all elements to True
        mask[k_index] = False # set test elements to False
        y_cross_val_train = y_single_jet_train[mask] # select only True elements (ie train elements)
        tx_cross_val_train = tx_single_jet_train_preprocessed[mask,:]
        
        # Compute the accuracies for each degree
        accuracies_train_by_fold[:,:,:,k], accuracies_test_by_fold[:,:,:,k] = cross_validation_one_fold_logistic_regularized\
            (y_cross_val_train, y_cross_val_test, tx_cross_val_train, tx_cross_val_test, \
                                 degrees, gammas, lambdas, len_kept_data, max_iters, True)
    
    # Compute the mean accuracies over the folds, for each degree
    mean_accuracies_train_by_deg = np.mean(accuracies_train_by_fold, axis=3)
    mean_accuracies_test_by_deg = np.mean(accuracies_test_by_fold, axis=3)
    
    # Get the index of the best accuracy in the testing set
    max_id_deg_test, max_id_gamma_test,max_id_lambda = \
        np.unravel_index(mean_accuracies_test_by_deg.argmax(), mean_accuracies_test_by_deg.shape)
    
    # Find the optimal degree and the corresponding accuracies in the training and testing sets
    best_deg = degrees[max_id_deg_test]
    best_gamma = gammas[max_id_gamma_test]
    best_lambda=lambdas[max_id_lambda]
    best_accuracy_test = mean_accuracies_test_by_deg[max_id_deg_test, max_id_gamma_test,max_id_lambda]
    corresponding_accuracy_train = mean_accuracies_train_by_deg[max_id_deg_test, max_id_gamma_test,max_id_lambda]
    
    print('Best accuracy test =', best_accuracy_test, 'with degree =', best_deg)
    print('Corresponding accuracy train =', corresponding_accuracy_train)
    
    return best_deg, best_gamma, best_lambda, best_accuracy_test, corresponding_accuracy_train                        


In [58]:
degrees = range(6,11)
gammas = np.logspace(-8,-2,6)
lambdas = np.logspace(-8,-2,6)

k_fold = 5
seed = 1
max_iters = 300

In [ ]:
mask_jets_train = split_jets_mask(tx_train)
mask_jets_test = split_jets_mask(tx_test)
len_mask = len(mask_jets_train)

y_predicted_train = np.zeros(len(y_train))
y_predicted_test = np.zeros(tx_test.shape[0])
best_degrees = np.zeros(len_mask)
best_gammas = np.zeros(len_mask)
best_lambdas=np.zeros(len_mask)

In [ ]:
for jet_id in range(len_mask):
    print('** Jet ', jet_id, '**')
    # SEPARATE THE WHOLE DATA SET TO GET ONLY THE PART THAT HAVE THE RIGHT NUMBER OF JETS
    tx_single_jet_train = tx_train[mask_jets_train[jet_id]]
    tx_single_jet_test = tx_test[mask_jets_test[jet_id]]
    y_single_jet_train = y_train[mask_jets_train[jet_id]]
    
    # CALL CROSS VALIDATION FOR A SINGLE JET ON TRAIN PART, FIND BEST DEG, BEST ACCURACY ON TESTING CROSS VAL
    best_deg, best_gamma, best_lambda, best_accuracy_test, corresponding_accuracy_train = \
        cross_validation_logistic_regularized(y_single_jet_train, tx_single_jet_train, degrees, gammas, lambdas, k_fold, seed, max_iters)
    
    # KEEP IN MEMORY THE BEST DEGREE FOR THIS JET
    best_degrees[jet_id] = best_deg
    best_gammas[jet_id] = best_gamma
    best_lambdas[jet_id]=best_lambda
    # PREPROCESS FULL TRAINING AND TESTING DATA
    tx_single_jet_train_preprocessed, len_kept_data, unique_cols = \
        preprocess_data(tx_single_jet_train, [], 'after')
    tx_single_jet_test_preprocessed = preprocess_data(tx_single_jet_test, unique_cols, 'after')[0]
    
    # ADD POWERS TO THE CHOSEN COLUMNS
    len_data = tx_single_jet_train_preprocessed.shape[1]
    tx_single_jet_train_preprocessed = add_powers(tx_single_jet_train_preprocessed, range(2,best_deg+1), \
                                                  range(len_kept_data))
    tx_single_jet_test_preprocessed = add_powers(tx_single_jet_test_preprocessed, range(2,best_deg+1), \
                                                 range(len_kept_data))
    tx_single_jet_train_preprocessed[:,len_data:] = standardize(tx_single_jet_train_preprocessed[:,len_data:])[0]
    tx_single_jet_test_preprocessed[:,len_data:] = standardize(tx_single_jet_test_preprocessed[:,len_data:])[0]
    
    # COMPUTE THE BEST WEIGHTS AND FULL ACCURACY ON TRAINING FULL SET - ONE JET
    initial_w = np.zeros(tx_single_jet_train_preprocessed.shape[1])
    weights, loss = reg_logistic_regression(y_single_jet_train, tx_single_jet_train_preprocessed , best_lambda, initial_w, max_iters, best_gamma)
    
    # COMPUTE THE PREDICTIONS ON THE FULL TESTING SET - SINGLE JET
    y_predicted_single_jet_train = predict_labels(weights, tx_single_jet_train_preprocessed)
    y_predicted_single_jet_test = predict_labels(weights, tx_single_jet_test_preprocessed)
    
    # ADD THE PREDICTIONS TO y_predicted_test AND y_predicted_train
    y_predicted_train[mask_jets_train[jet_id]] = y_predicted_single_jet_train
    y_predicted_test[mask_jets_test[jet_id]] = y_predicted_single_jet_test
    
    # COMPUTE THE ACCURACY train ON JET
    accuracy_train_single_jet = np.sum(y_predicted_single_jet_train == y_single_jet_train)/len(y_single_jet_train)
    
    # PRINT ACCURACY train ON JET
    print('Accuracy full train on jet', jet_id, '=', accuracy_train_single_jet)

** Jet  0 **
--- Fold 0 ---
++ Degree 6 ++
>> Gamma 1e-08 <<
>> Lambda 1e-08 <<
>> Lambda 1.58489319246e-07 <<
>> Lambda 2.51188643151e-06 <<
>> Lambda 3.98107170553e-05 <<
>> Lambda 0.00063095734448 <<
>> Lambda 0.01 <<
>> Gamma 1.58489319246e-07 <<
>> Lambda 1e-08 <<
>> Lambda 1.58489319246e-07 <<
>> Lambda 2.51188643151e-06 <<
>> Lambda 3.98107170553e-05 <<
>> Lambda 0.00063095734448 <<
>> Lambda 0.01 <<
>> Gamma 2.51188643151e-06 <<
>> Lambda 1e-08 <<
>> Lambda 1.58489319246e-07 <<
>> Lambda 2.51188643151e-06 <<
>> Lambda 3.98107170553e-05 <<
>> Lambda 0.00063095734448 <<
>> Lambda 0.01 <<
>> Gamma 3.98107170553e-05 <<
>> Lambda 1e-08 <<
>> Lambda 1.58489319246e-07 <<
>> Lambda 2.51188643151e-06 <<
>> Lambda 3.98107170553e-05 <<
>> Lambda 0.00063095734448 <<
>> Lambda 0.01 <<
>> Gamma 0.00063095734448 <<
>> Lambda 1e-08 <<
>> Lambda 1.58489319246e-07 <<
>> Lambda 2.51188643151e-06 <<
>> Lambda 3.98107170553e-05 <<
>> Lambda 0.00063095734448 <<
>> Lambda 0.01 <<
>> Gamma 0.01 <<
>> 

>> Lambda 2.51188643151e-06 <<
>> Lambda 3.98107170553e-05 <<
>> Lambda 0.00063095734448 <<
>> Lambda 0.01 <<
>> Gamma 2.51188643151e-06 <<
>> Lambda 1e-08 <<
>> Lambda 1.58489319246e-07 <<
>> Lambda 2.51188643151e-06 <<
>> Lambda 3.98107170553e-05 <<
>> Lambda 0.00063095734448 <<
>> Lambda 0.01 <<
>> Gamma 3.98107170553e-05 <<
>> Lambda 1e-08 <<
>> Lambda 1.58489319246e-07 <<
>> Lambda 2.51188643151e-06 <<
>> Lambda 3.98107170553e-05 <<
>> Lambda 0.00063095734448 <<
>> Lambda 0.01 <<
>> Gamma 0.00063095734448 <<
>> Lambda 1e-08 <<
>> Lambda 1.58489319246e-07 <<
>> Lambda 2.51188643151e-06 <<
>> Lambda 3.98107170553e-05 <<
>> Lambda 0.00063095734448 <<
>> Lambda 0.01 <<
>> Gamma 0.01 <<
>> Lambda 1e-08 <<
>> Lambda 1.58489319246e-07 <<
>> Lambda 2.51188643151e-06 <<
>> Lambda 3.98107170553e-05 <<
>> Lambda 0.00063095734448 <<
>> Lambda 0.01 <<
++ Degree 9 ++
>> Gamma 1e-08 <<
>> Lambda 1e-08 <<
>> Lambda 1.58489319246e-07 <<
>> Lambda 2.51188643151e-06 <<
>> Lambda 3.98107170553e-05 <<

>> Lambda 0.01 <<
>> Gamma 3.98107170553e-05 <<
>> Lambda 1e-08 <<
>> Lambda 1.58489319246e-07 <<
>> Lambda 2.51188643151e-06 <<
>> Lambda 3.98107170553e-05 <<
>> Lambda 0.00063095734448 <<
>> Lambda 0.01 <<
>> Gamma 0.00063095734448 <<
>> Lambda 1e-08 <<
>> Lambda 1.58489319246e-07 <<
>> Lambda 2.51188643151e-06 <<
>> Lambda 3.98107170553e-05 <<
>> Lambda 0.00063095734448 <<
>> Lambda 0.01 <<
>> Gamma 0.01 <<
>> Lambda 1e-08 <<
>> Lambda 1.58489319246e-07 <<
>> Lambda 2.51188643151e-06 <<
>> Lambda 3.98107170553e-05 <<
>> Lambda 0.00063095734448 <<
>> Lambda 0.01 <<
--- Fold 3 ---
++ Degree 6 ++
>> Gamma 1e-08 <<
>> Lambda 1e-08 <<
>> Lambda 1.58489319246e-07 <<
>> Lambda 2.51188643151e-06 <<
>> Lambda 3.98107170553e-05 <<
>> Lambda 0.00063095734448 <<
>> Lambda 0.01 <<
>> Gamma 1.58489319246e-07 <<
>> Lambda 1e-08 <<
>> Lambda 1.58489319246e-07 <<
>> Lambda 2.51188643151e-06 <<
>> Lambda 3.98107170553e-05 <<
>> Lambda 0.00063095734448 <<
>> Lambda 0.01 <<
>> Gamma 2.51188643151e-06 <

>> Lambda 2.51188643151e-06 <<
>> Lambda 3.98107170553e-05 <<
>> Lambda 0.00063095734448 <<
>> Lambda 0.01 <<
>> Gamma 0.01 <<
>> Lambda 1e-08 <<
>> Lambda 1.58489319246e-07 <<
>> Lambda 2.51188643151e-06 <<
>> Lambda 3.98107170553e-05 <<
>> Lambda 0.00063095734448 <<
>> Lambda 0.01 <<
++ Degree 8 ++
>> Gamma 1e-08 <<
>> Lambda 1e-08 <<
>> Lambda 1.58489319246e-07 <<
>> Lambda 2.51188643151e-06 <<
>> Lambda 3.98107170553e-05 <<
>> Lambda 0.00063095734448 <<
>> Lambda 0.01 <<
>> Gamma 1.58489319246e-07 <<
>> Lambda 1e-08 <<
>> Lambda 1.58489319246e-07 <<
>> Lambda 2.51188643151e-06 <<
>> Lambda 3.98107170553e-05 <<
>> Lambda 0.00063095734448 <<
>> Lambda 0.01 <<
>> Gamma 2.51188643151e-06 <<
>> Lambda 1e-08 <<
>> Lambda 1.58489319246e-07 <<
>> Lambda 2.51188643151e-06 <<
>> Lambda 3.98107170553e-05 <<
>> Lambda 0.00063095734448 <<
>> Lambda 0.01 <<
>> Gamma 3.98107170553e-05 <<
>> Lambda 1e-08 <<
>> Lambda 1.58489319246e-07 <<
>> Lambda 2.51188643151e-06 <<
>> Lambda 3.98107170553e-05 <

>> Lambda 1.58489319246e-07 <<
>> Lambda 2.51188643151e-06 <<
>> Lambda 3.98107170553e-05 <<
>> Lambda 0.00063095734448 <<
>> Lambda 0.01 <<
++ Degree 10 ++
>> Gamma 1e-08 <<
>> Lambda 1e-08 <<
>> Lambda 1.58489319246e-07 <<
>> Lambda 2.51188643151e-06 <<
>> Lambda 3.98107170553e-05 <<
>> Lambda 0.00063095734448 <<
>> Lambda 0.01 <<
>> Gamma 1.58489319246e-07 <<
>> Lambda 1e-08 <<
>> Lambda 1.58489319246e-07 <<
>> Lambda 2.51188643151e-06 <<
>> Lambda 3.98107170553e-05 <<
>> Lambda 0.00063095734448 <<
>> Lambda 0.01 <<
>> Gamma 2.51188643151e-06 <<
>> Lambda 1e-08 <<
>> Lambda 1.58489319246e-07 <<
>> Lambda 2.51188643151e-06 <<
>> Lambda 3.98107170553e-05 <<
>> Lambda 0.00063095734448 <<
>> Lambda 0.01 <<
>> Gamma 3.98107170553e-05 <<
>> Lambda 1e-08 <<
>> Lambda 1.58489319246e-07 <<
>> Lambda 2.51188643151e-06 <<
>> Lambda 3.98107170553e-05 <<
>> Lambda 0.00063095734448 <<
>> Lambda 0.01 <<
>> Gamma 0.00063095734448 <<
>> Lambda 1e-08 <<
>> Lambda 1.58489319246e-07 <<
>> Lambda 2.5118

>> Lambda 0.00063095734448 <<
>> Lambda 0.01 <<
>> Gamma 1.58489319246e-07 <<
>> Lambda 1e-08 <<
>> Lambda 1.58489319246e-07 <<
>> Lambda 2.51188643151e-06 <<
>> Lambda 3.98107170553e-05 <<
>> Lambda 0.00063095734448 <<
>> Lambda 0.01 <<
>> Gamma 2.51188643151e-06 <<
>> Lambda 1e-08 <<
>> Lambda 1.58489319246e-07 <<
>> Lambda 2.51188643151e-06 <<
>> Lambda 3.98107170553e-05 <<
>> Lambda 0.00063095734448 <<
>> Lambda 0.01 <<
>> Gamma 3.98107170553e-05 <<
>> Lambda 1e-08 <<
>> Lambda 1.58489319246e-07 <<
>> Lambda 2.51188643151e-06 <<
>> Lambda 3.98107170553e-05 <<
>> Lambda 0.00063095734448 <<
>> Lambda 0.01 <<
>> Gamma 0.00063095734448 <<
>> Lambda 1e-08 <<
>> Lambda 1.58489319246e-07 <<
>> Lambda 2.51188643151e-06 <<
>> Lambda 3.98107170553e-05 <<
>> Lambda 0.00063095734448 <<
>> Lambda 0.01 <<
>> Gamma 0.01 <<
>> Lambda 1e-08 <<
>> Lambda 1.58489319246e-07 <<
>> Lambda 2.51188643151e-06 <<
>> Lambda 3.98107170553e-05 <<
>> Lambda 0.00063095734448 <<
>> Lambda 0.01 <<
++ Degree 8 ++
>

>> Lambda 1.58489319246e-07 <<
>> Lambda 2.51188643151e-06 <<
>> Lambda 3.98107170553e-05 <<
>> Lambda 0.00063095734448 <<
>> Lambda 0.01 <<
>> Gamma 3.98107170553e-05 <<
>> Lambda 1e-08 <<
>> Lambda 1.58489319246e-07 <<
>> Lambda 2.51188643151e-06 <<
>> Lambda 3.98107170553e-05 <<
>> Lambda 0.00063095734448 <<
>> Lambda 0.01 <<
>> Gamma 0.00063095734448 <<
>> Lambda 1e-08 <<
>> Lambda 1.58489319246e-07 <<
>> Lambda 2.51188643151e-06 <<
>> Lambda 3.98107170553e-05 <<
>> Lambda 0.00063095734448 <<
>> Lambda 0.01 <<
>> Gamma 0.01 <<
>> Lambda 1e-08 <<
>> Lambda 1.58489319246e-07 <<
>> Lambda 2.51188643151e-06 <<
>> Lambda 3.98107170553e-05 <<
>> Lambda 0.00063095734448 <<
>> Lambda 0.01 <<
++ Degree 10 ++
>> Gamma 1e-08 <<
>> Lambda 1e-08 <<
>> Lambda 1.58489319246e-07 <<
>> Lambda 2.51188643151e-06 <<
>> Lambda 3.98107170553e-05 <<
>> Lambda 0.00063095734448 <<
>> Lambda 0.01 <<
>> Gamma 1.58489319246e-07 <<
>> Lambda 1e-08 <<
>> Lambda 1.58489319246e-07 <<
>> Lambda 2.51188643151e-06 <

>> Lambda 0.01 <<
>> Gamma 3.98107170553e-05 <<
>> Lambda 1e-08 <<
>> Lambda 1.58489319246e-07 <<
>> Lambda 2.51188643151e-06 <<
>> Lambda 3.98107170553e-05 <<
>> Lambda 0.00063095734448 <<
>> Lambda 0.01 <<
>> Gamma 0.00063095734448 <<
>> Lambda 1e-08 <<
>> Lambda 1.58489319246e-07 <<
>> Lambda 2.51188643151e-06 <<
>> Lambda 3.98107170553e-05 <<
>> Lambda 0.00063095734448 <<
>> Lambda 0.01 <<
>> Gamma 0.01 <<
>> Lambda 1e-08 <<
>> Lambda 1.58489319246e-07 <<


In [ ]:
# CREATE CSV SUBMISSION
#create_csv_submission(ids_test, y_predicted_test, 'output/trial.csv')

# COMPUTE ACCURACY ON FULL train
total_accuracy_train = np.sum(y_predicted_train == y_train)/len(y_train)*100
print('Total accuracy train =', total_accuracy_train, 'with degrees =', best_degrees, \
      ', gammas =', best_gammas, 'and lambdas =', best_lambdas)

In [ ]:
total_accuracy_test = np.sum(y_predicted_test == y_test)/len(y_test)*100
print('Total accuracy test =', total_accuracy_test, 'with degrees =', best_degrees)

In [ ]:
np.sum(y_predicted_test==-1)

In [ ]:
np.sum(y_predicted_test==1)

In [ ]:
y_predicted_test[:200]